Input: adjacency matrix


In [90]:
import json

with open('recipes-1.JSON') as recipes:
    data = json.load(recipes)
# print(data)

ingrs_per_recipe = []
for obj in data:
    ingrs_per_recipe.append(obj["ingredients"])
print(ingrs_per_recipe)

[['ponzu', 'shallot', 'mustard', 'olive', 'olive oil', 'wasabi', 'steak', 'tea', 'cucumber', 'salt'], ['rice', 'vinegar', 'rice vinegar', 'lemon', 'lime', 'lime juice', 'soy sauce'], ['paprika', 'sugar', 'brown sugar', 'salt', 'mustard', 'seed', 'garlic', 'garlic powder', 'onion', 'onion powder', 'oregano', 'coriander', 'cumin', 'tea', 'pepper', 'red pepper flakes', 'red pepper', 'ketchup', 'molasses', 'worcestershire sauce', 'apple', 'cider', 'vinegar', 'apple cider vinegar', 'pork'], ['flour', 'sugar', 'salt', 'tea', 'butter', 'water'], ['pepper', 'poblano', 'poblano pepper', 'potato', 'salt', 'vinegar', 'white vinegar', 'onion', 'red onion', 'tea', 'thyme', 'cilantro', 'parsley'], ['lemon', 'lemons', 'sugar', 'tea'], ['lemon', 'lemons', 'sugar', 'water'], ['water', 'tea'], ['bean', 'beans', 'pinto bean', 'pinto beans', 'salt', 'bacon', 'onion', 'chili', 'seed', 'seeds', 'clove', 'cloves', 'garlic', 'tomato', 'tomatoes', 'chicken', 'epazote', 'cilantro'], ['coffee', 'tea', 'chocolate

In [129]:
with open('foodList.txt') as f:
        ing_list = f.readlines()
ing_list = [ing[:-1].lower() for ing in ing_list]
print(ing_list)


['abalone', 'absinthe', 'achar', 'ackee', 'adobo', 'advieh', 'agar', 'aioli', 'ajowan', 'ale', 'alitame', 'alligator', 'allspice', 'almond', 'almonds', 'almond bark', 'almond butter', 'alum', 'amaranth', 'amaretti', 'amaretto', 'amla', 'anchovette', 'anchovy', 'anchovies', 'angelica', 'anise', 'apple', 'apples', 'apple butter', 'apple jelly', 'apricot', 'apricots', 'apricot jelly', 'apricot jam', 'aquavit', 'arak', 'arrowroot', 'artichoke', 'arugula', 'asadero', 'asafetida', 'asiago cheese', 'asparagus', 'aspic', 'avocado', 'avocados', 'baba', 'babaco', 'babka', 'bacon', 'bagel', 'bagels', 'bagna cauda', 'bagoong', 'baguette', 'baguettes', 'baharat', 'baklava', 'banger', 'banana', 'bananas', 'bard', 'barley', 'baron', 'basil', 'bass', 'beat', 'bechamel', 'beef', 'beer', 'beet', 'beets', 'bento', 'berbere', 'berry', 'berries', 'biltong', 'bisque', 'blackberry', 'blackberries', 'beurre blanc', 'blueberry', 'blueberries', 'boerewors', 'bok choy', 'borage', 'borscht', 'bottarga', 'bouillon

In [180]:
import pandas as pd
import itertools as iter

# with open("foodList.txt")
# ingredients = pd.read_table('foodList.txt', skip_blank_lines=True)
# print(type(ingredients))
columns = ing_list
df = pd.DataFrame(0, index=columns, columns=columns)

for ingredient_list in ingrs_per_recipe:
#     print('ingredient_list:', ingredient_list)
    for pair in list(iter.combinations(ingredient_list,2)):
#         print(pair)
        r = pair[0]
        c = pair[1]
#         print(curr_value)
#         print('curr_value', curr_value+1)
        curr_value = df.loc[r, c]
        new_value = curr_value + 1
        df.ix[r, c] = new_value
#         print(df.loc[r][c])
# print(df.loc['fennel',:].sum())
print(df.loc['fennel',:])
        
        
# for recipe in ingrs_per_recipe[0]:
#     print(recipe)
#     for pair in list(iter.combinations(recipe, 2)): #nC2
#         print(pair)
# #         r = pair[0]
# #         c = pair[1]
# #         df.loc[r, c] += 1

# print(df)

        abalone  absinthe  achar  ackee  adobo  advieh  agar  aioli  ajowan  \
fennel        0         0      0      0      0       0     0      0       0   
fennel        0         0      0      0      0       0     0      0       0   

        ale       ...        won ton  worcestershire sauce  mushu  lumpia  \
fennel    0       ...              0                     1      0       0   
fennel    0       ...              0                     1      0       0   

        xanthan gum  cornish yarg cheese  hundred-year egg  yeast  \
fennel            0                    0                 0      0   
fennel            0                    0                 0      0   

        champagne yeast  brewer's yeast  
fennel                0               0  
fennel                0               0  

[2 rows x 1025 columns]


In [ ]:
# Dijkstra's algorithm
# 1. Initializations
# 2. S = [];
# 3. Q = G.vertices
# 4. while Q.length != 0
#       u = EXTRACT-MIN

def Dijkstras(G, )

In [102]:
x = 10
print('x', x)

x 10


In [128]:
print(df.loc['Ponzu']['Ponzu'])

0


In [172]:
columns = ['a a', 'b b', 'c c']
df2 = pd.DataFrame(0, index=columns, columns=columns)
# print(df2)

r = 'a a'
c = 'b b'
df2.ix[r, c]+=1
print(df2)
df2.ix[r, c]+=1
print(df2)
c = 'c'
df2.ix[r, c]+=1
print(df2)
df2.ix[r, c]+=1
print(df2)
df2.ix[r, c]+=1
print(df2)
df2.ix[r, c]+=1
print(df2)
df2.ix[r, c]+=1
print(df2)
df2.ix[r, c]+=1
print(df2)
df2.ix[r, c]+=1
print(df2)

     a a  b b  c c
a a    0    1    0
b b    0    0    0
c c    0    0    0
     a a  b b  c c
a a    0    2    0
b b    0    0    0
c c    0    0    0


KeyError: 'c'